In [9]:
from ttml.tt_cross import estimator_to_tt_cross
import numpy as np

def f(x):
    return np.sin(np.sum(x, axis=1))

# Same thresholds for every feature: [0,0.2,0.4,0.6,0.8]
thresholds = [np.linspace(0, 1, 11)] * 5

tt = estimator_to_tt_cross(f, thresholds)
tt.gather(np.array([[2, 2, 2, 2, 2]])) - np.sin(1)

array([1.11022302e-16])

In [10]:
tt.sing_vals()

[array([2.62706082e+02, 1.10057236e+02, 1.01854161e-13, 7.11972404e-14,
        4.37889373e-14]),
 array([2.50489642e+02, 1.35580309e+02, 5.40010485e-14, 2.48204966e-14,
        1.87124926e-14]),
 array([2.50489642e+02, 1.35580309e+02, 3.23567263e-14, 2.47347881e-14,
        1.88056492e-14]),
 array([2.62706082e+02, 1.10057236e+02, 2.87919777e-14, 2.27094898e-14,
        1.03424501e-14])]

In [11]:
tt2 = estimator_to_tt_cross(f, thresholds, max_rank=2)

tt2

<TensorTrain of order 5 with outer dimensions (11, 11, 11, 11, 11), TT-rank (2, 2, 2, 2), and orthogonalized at mode 4>

In [12]:
(tt - tt2).norm()

4.916953631940722e-13

In [13]:
from sklearn.ensemble import RandomForestRegressor

X = np.random.normal(size=(1000, 5))
y = np.exp(np.sum(X, axis=1))

forest = RandomForestRegressor()
forest.fit(X, y)

thresholds = [np.linspace(0, 1, 11)] * 5

tt = estimator_to_tt_cross(forest.predict, thresholds, max_rank=2)
tt

<TensorTrain of order 5 with outer dimensions (11, 11, 11, 11, 11), TT-rank (2, 2, 2, 2), and orthogonalized at mode 4>

In [14]:
tt = estimator_to_tt_cross(forest.predict, thresholds, method='dmrg')

In [15]:
import tensorly as tl
from tensorly.contrib.decomposition import tensor_train_cross

In [16]:
tensor = tl.tensor(np.arange(5**3).reshape(5,5,5))

In [17]:
rank = [1, 3, 3, 1]

In [18]:
factors = tensor_train_cross(tensor, rank)

In [19]:
print(factors[0])

[[[ 24   7   0]
  [ 49  32  25]
  [ 74  57  50]
  [ 99  82  75]
  [124 107 100]]]


In [26]:
import tensorly as tl
from tensorly import tt_to_tensor
import numpy as np

def tensor_train_cross(input_tensor, rank, tol=1e-4, n_iter_max=100, random_state=None):
    """TT (tensor-train) decomposition via cross-approximation (TTcross) [1]

    Decomposes `input_tensor` into a sequence of order-3 tensors of given rank. (factors/cores)
    Rather than directly decompose the whole tensor, we sample fibers based on skeleton decomposition.
    We initialize a random tensor-train and sweep from left to right and right to left.
    On each core, we shape the core as a matrix and choose the fibers indices by finding maximum-volume submatrix and update the core.

    * Advantage: faster
        The main advantage of TTcross is that it doesn't need to evaluate all the entries of the tensor.
        For a tensor_shape^tensor_order tensor, SVD needs O(tensor_shape^tensor_order) runtime, but TTcross' runtime is linear in tensor_shape and tensor_order, which makes it feasible in high dimension.
    * Disadvantage: less accurate
        TTcross may underestimate the error, since it only evaluates partial entries of the tensor.
        Besides, in contrast to its practical fast performance, there is no theoretical guarantee of it convergence.

    Parameters
    ----------
    input_tensor : tensorly.tensor
            The tensor to decompose.
    rank : {int, int list}
            maximum allowable TT rank of the factors
            if int, then this is the same for all the factors
            if int list, then rank[k] is the rank of the kth factor
    tol : float
            accuracy threshold for outer while-loop
    n_iter_max : int
            maximum iterations of outer while-loop (the 'crosses' or 'sweeps' sampled)
    random_state : {None, int, np.random.RandomState}

    Returns
    -------
    factors : TT factors
              order-3 tensors of the TT decomposition

    Examples
    --------

    Generate a 5^3 tensor, and decompose it into tensor-train of 3 factors, with rank = [1,3,3,1]

    >>> tensor = tl.tensor(np.arange(5**3).reshape(5,5,5))
    >>> rank = [1, 3, 3, 1]
    >>> factors = tensor_train_cross(tensor, rank)
    >>> # print the first core:
    >>> print(factors[0])
    [[[ 24.   0.   4.]
      [ 49.  25.  29.]
      [ 74.  50.  54.]
      [ 99.  75.  79.]
      [124. 100. 104.]]]

    Notes
    -----
    Pseudo-code [2]:

    1. Initialization tensor_order cores and column indices

    2. while (error > tol)

    3. update the tensor-train from left to right:

       .. code:: python

           for Core 1 to Core tensor_order:
           approximate the skeleton-decomposition by QR and maxvol

    4. update the tensor-train from right to left:

       .. code:: python

            for Core tensor_order to Core 1
                approximate the skeleton-decomposition by QR and maxvol

    5. end while

    Acknowledgement: the main body of the code is modified based on TensorToolbox by Daniele Bigoni.

    References
    ----------
    .. [1] Ivan Oseledets and Eugene Tyrtyshnikov.  Tt-cross approximation for multidimensional arrays.
            LinearAlgebra and its Applications, 432(1):70–88, 2010.
    .. [2] Sergey Dolgov and Robert Scheichl. A hybrid alternating least squares–tt cross algorithm for parametricpdes.
            arXiv preprint arXiv:1707.04562, 2017.
    """

    # Check user input for errors
    tensor_shape = tl.shape(input_tensor)
    tensor_order = tl.ndim(input_tensor)

    if isinstance(rank, int):
        rank = [rank] * (tensor_order + 1)
    elif tensor_order + 1 != len(rank):
        message = (
            "Provided incorrect number of ranks. Should verify "
            + f"len(rank) == tl.ndim(tensor)+1, but len(rank) = {len(rank)} "
            + f"while tl.ndim(tensor) + 1  = {tensor_order}"
        )
        raise (ValueError(message))

    # Make sure iter's not a tuple but a list
    rank = list(rank)

    # Initialize rank
    if rank[0] != 1:
        message = f"Provided rank[0] == {rank[0]} but boundary conditions dictate rank[0] == rank[-1] == 1."
        raise ValueError(message)
    if rank[-1] != 1:
        message = f"Provided rank[-1] == {rank[-1]} but boundary conditions dictate rank[0] == rank[-1] == 1."
        raise ValueError(message)

    # list col_idx: column indices (right indices) for skeleton-decomposition: indicate which columns used in each core.
    # list row_idx: row indices    (left indices)  for skeleton-decomposition: indicate which rows used in each core.

    # Initialize indice: random selection of column indices
    rng = tl.check_random_state(random_state)

    col_idx = [None] * tensor_order
    for k_col_idx in range(tensor_order - 1):
        col_idx[k_col_idx] = []
        for i in range(rank[k_col_idx + 1]):
            newidx = tuple(
                [
                    rng.randint(tensor_shape[j])
                    for j in range(k_col_idx + 1, tensor_order)
                ]
            )
            while newidx in col_idx[k_col_idx]:
                newidx = tuple(
                    [
                        rng.randint(tensor_shape[j])
                        for j in range(k_col_idx + 1, tensor_order)
                    ]
                )

            col_idx[k_col_idx].append(newidx)

    # Initialize the cores of tensor-train
    factor_old = [
        tl.zeros((rank[k], tensor_shape[k], rank[k + 1]), **tl.context(input_tensor))
        for k in range(tensor_order)
    ]
    factor_new = [
        tl.tensor(
            rng.random_sample((rank[k], tensor_shape[k], rank[k + 1])),
            **tl.context(input_tensor),
        )
        for k in range(tensor_order)
    ]

    iter = 0

    error = tl.norm(tt_to_tensor(factor_old) - tt_to_tensor(factor_new), 2)
    threshold = tol * tl.norm(tt_to_tensor(factor_new), 2)
    for iter in range(n_iter_max):
        if error < threshold:
            break

        factor_old = factor_new
        factor_new = [None for i in range(tensor_order)]

        ######################################
        # left-to-right step
        left_to_right_fiberlist = []
        # list row_idx: list of (tensor_order-1) of lists of left indices
        row_idx = [[()]]
        if iter == 0:
            print(row_idx)
            print(col_idx)
        for k in range(tensor_order - 1):
            (next_row_idx, fibers_list) = left_right_ttcross_step(
                input_tensor, k, rank, row_idx, col_idx
            )
            # update row indices
            left_to_right_fiberlist.extend(fibers_list)
            row_idx.append(next_row_idx)

        # end left-to-right step
        ###############################################

        ###############################################
        # right-to-left step
        right_to_left_fiberlist = []
        # list col_idx: list (tensor_order-1) of lists of right indices
        col_idx = [None] * tensor_order
        col_idx[-1] = [()]
        for k in range(tensor_order, 1, -1):
            (next_col_idx, fibers_list, Q_skeleton) = right_left_ttcross_step(
                input_tensor, k, rank, row_idx, col_idx
            )
            # update col indices
            right_to_left_fiberlist.extend(fibers_list)
            col_idx[k - 2] = next_col_idx

            # Compute cores
            try:
                factor_new[k - 1] = tl.transpose(Q_skeleton)
                factor_new[k - 1] = tl.reshape(
                    factor_new[k - 1], (rank[k - 1], tensor_shape[k - 1], rank[k])
                )
            except:
                # The rank should not be larger than the input tensor's size
                raise (
                    ValueError(
                        "The rank is too large compared to the size of the tensor. Try with small rank."
                    )
                )

        # Add the last core
        idx = (slice(None, None, None),) + tuple(zip(*col_idx[0]))

        core = input_tensor[idx]
        core = tl.reshape(core, (tensor_shape[0], 1, rank[1]))
        core = tl.transpose(core, (1, 0, 2))

        factor_new[0] = core

        # end right-to-left step
        ################################################

        # check the error for while-loop
        error = tl.norm(tt_to_tensor(factor_old) - tt_to_tensor(factor_new), 2)
        threshold = tol * tl.norm(tt_to_tensor(factor_new), 2)

    # check convergence
    if iter >= n_iter_max:
        raise ValueError("Maximum number of iterations reached.")
    if tl.norm(tt_to_tensor(factor_old) - tt_to_tensor(factor_new), 2) > tol * tl.norm(
        tt_to_tensor(factor_new), 2
    ):
        raise ValueError("Low Rank Approximation algorithm did not converge.")

    return factor_new




def left_right_ttcross_step(input_tensor, k, rank, row_idx, col_idx):
    """Compute the next (right) core's row indices by QR decomposition.

    For the current Tensor train core, we use the row indices and col indices to extract the entries from the input tensor
    and compute the next core's row indices by QR and max volume algorithm.

    Parameters
    ----------

    k: int
            the actual sweep iteration
    rank: list of int
            list of upper ranks (tensor_order)
    row_idx: list of list of int
            list of (tensor_order-1) of lists of left indices
    col_idx: list of list of int
            list of (tensor_order-1) of lists of right indices

    Returns
    -------
    next_row_idx : list of int
            the list of new row indices,
    fibers_list : list of slice
            the used fibers,
    Q_skeleton : matrix
            approximation of Q as product of Q and inverse of its maximum volume submatrix
    """

    tensor_shape = tl.shape(input_tensor)
    tensor_order = tl.ndim(input_tensor)
    fibers_list = []

    # Extract fibers according to the row and col indices
    for i in range(rank[k]):
        for j in range(rank[k + 1]):
            fiber = row_idx[k][i] + (slice(None, None, None),) + col_idx[k][j]
            fibers_list.append(fiber)
    if k == 0:  # Is[k] will be empty
        idx = (slice(None, None, None),) + tuple(zip(*col_idx[k]))
    else:
        idx = [[] for i in range(tensor_order)]
        for lidx in row_idx[k]:
            for ridx in col_idx[k]:
                for j, jj in enumerate(lidx):
                    idx[j].append(jj)
                for j, jj in enumerate(ridx):
                    idx[len(lidx) + 1 + j].append(jj)
        idx[k] = slice(None, None, None)
        idx = tuple(idx)

    # Extract the core
    core = input_tensor[idx]
    # shape the core as a 3-tensor_order cube
    if k == 0:
        core = tl.reshape(core, (tensor_shape[k], rank[k], rank[k + 1]))
        core = tl.transpose(core, (1, 0, 2))
    else:
        core = tl.reshape(core, (rank[k], rank[k + 1], tensor_shape[k]))
        core = tl.transpose(core, (0, 2, 1))

    # merge r_k and n_k, get a matrix
    core = tl.reshape(core, (rank[k] * tensor_shape[k], rank[k + 1]))

    # Compute QR decomposition
    (Q, R) = tl.qr(core)

    # Maxvol
    (I, _) = maxvol(Q)

    # Retrive indices in folded tensor
    new_idx = [
        np.unravel_index(idx, [rank[k], tensor_shape[k]]) for idx in I
    ]  # First retrive idx in folded core
    next_row_idx = [
        row_idx[k][ic[0]] + (ic[1],) for ic in new_idx
    ]  # Then reconstruct the idx in the tensor

    return (next_row_idx, fibers_list)


def right_left_ttcross_step(input_tensor, k, rank, row_idx, col_idx):
    """Compute the next (left) core's col indices by QR decomposition.

    For the current Tensor train core, we use the row indices and col indices to extract the entries from the input tensor
    and compute the next core's col indices by QR and max volume algorithm.

    Parameters
    ----------

    k: int
            the actual sweep iteration
    rank: list of int
            list of upper rank (tensor_order)
    row_idx: list of list of int
            list of (tensor_order-1) of lists of left indices
    col_idx: list of list of int
            list of (tensor_order-1) of lists of right indices

    Returns
    -------
    next_col_idx : list of int
            the list of new col indices,
    fibers_list : list of slice
            the used fibers,
    Q_skeleton : matrix
            approximation of Q as product of Q and inverse of its maximum volume submatrix
    """

    tensor_shape = tl.shape(input_tensor)
    tensor_order = tl.ndim(input_tensor)
    fibers_list = []

    # Extract fibers
    for i in range(rank[k - 1]):
        for j in range(rank[k]):
            fiber = row_idx[k - 1][i] + (slice(None, None, None),) + col_idx[k - 1][j]
            fibers_list.append(fiber)

    if k == tensor_order:  # Is[k] will be empty
        idx = tuple(zip(*row_idx[k - 1])) + (slice(None, None, None),)
    else:
        idx = [[] for i in range(tensor_order)]
        for lidx in row_idx[k - 1]:
            for ridx in col_idx[k - 1]:
                for j, jj in enumerate(lidx):
                    idx[j].append(jj)
                for j, jj in enumerate(ridx):
                    idx[len(lidx) + 1 + j].append(jj)
        idx[k - 1] = slice(None, None, None)
        idx = tuple(idx)

    core = input_tensor[idx]
    # shape the core as a 3-tensor_order cube
    core = tl.reshape(core, (rank[k - 1], rank[k], tensor_shape[k - 1]))
    core = tl.transpose(core, (0, 2, 1))
    # merge n_{k-1} and r_k, get a matrix
    core = tl.reshape(core, (rank[k - 1], tensor_shape[k - 1] * rank[k]))
    core = tl.transpose(core)

    # Compute QR decomposition
    (Q, R) = tl.qr(core)
    # Maxvol
    (J, Q_inv) = maxvol(Q)
    Q_inv = tl.tensor(Q_inv)
    Q_skeleton = tl.dot(Q, Q_inv)

    # Retrive indices in folded tensor
    new_idx = [
        np.unravel_index(idx, [tensor_shape[k - 1], rank[k]]) for idx in J
    ]  # First retrive idx in folded core
    next_col_idx = [
        (jc[0],) + col_idx[k - 1][jc[1]] for jc in new_idx
    ]  # Then reconstruct the idx in the tensor

    return (next_col_idx, fibers_list, Q_skeleton)


def maxvol(A):
    """Find the rxr submatrix of maximal volume in A(nxr), n>=r

    We want to decompose matrix A as `A = A[:,J] * (A[I,J])^-1 * A[I,:]`.
    This algorithm helps us find this submatrix A[I,J] from A, which has the largest determinant.
    We greedily find vector of max norm, and subtract its projection from the rest of rows.

    Parameters
    ----------

    A: matrix
        The matrix to find maximal volume

    Returns
    -------
    row_idx: list of int
        is the list or rows of A forming the matrix with maximal volume,
    A_inv: matrix
        is the inverse of the matrix with maximal volume.

    References
    ----------
    S. A. Goreinov, I. V. Oseledets, D. V. Savostyanov, E. E. Tyrtyshnikov, N. L. Zamarashkin.
    How to find a good submatrix.Goreinov, S. A., et al.
    Matrix Methods: Theory, Algorithms and Applications: Dedicated to the Memory of Gene Golub. 2010. 247-256.

    Ali Çivril, Malik Magdon-Ismail
    On selecting a maximum volume sub-matrix of a matrix and related problems
    Theoretical Computer Science. Volume 410, Issues 47–49, 6 November 2009, Pages 4801-4811
    """

    (n, r) = tl.shape(A)

    # The index of row of the submatrix
    row_idx = tl.zeros(r, dtype=tl.int64)

    # Rest of rows / unselected rows
    rest_of_rows = tl.tensor(list(range(n)), dtype=tl.int64)

    # Find r rows iteratively
    i = 0
    A_new = A
    while i < r:
        mask = list(range(tl.shape(A_new)[0]))
        # Compute the square of norm of each row
        rows_norms = tl.sum(A_new**2, axis=1)

        # If there is only one row of A left, let's just return it.
        if tl.shape(rows_norms) == ():
            row_idx[i] = rest_of_rows
            break

        # If a row is 0, we delete it.
        if any(rows_norms == 0):
            zero_idx = tl.argmin(rows_norms, axis=0)
            mask.pop(zero_idx)
            rest_of_rows = rest_of_rows[mask]
            A_new = A_new[mask, :]
            continue

        # Find the row of max norm
        max_row_idx = tl.argmax(rows_norms, axis=0)
        max_row = A[rest_of_rows[max_row_idx], :]

        # Compute the projection of max_row to other rows
        # projection a to b is computed as: <a,b> / sqrt(|a|*|b|)
        projection = tl.dot(A_new, tl.transpose(max_row))
        normalization = tl.sqrt(rows_norms[max_row_idx] * rows_norms)
        # make sure normalization vector is of the same shape of projection
        normalization = tl.reshape(normalization, tl.shape(projection))
        projection = projection / normalization

        # Subtract the projection from A_new:  b <- b - a * projection
        A_new = A_new - A_new * tl.reshape(projection, (tl.shape(A_new)[0], 1))

        # Delete the selected row
        mask.pop(tl.to_numpy(max_row_idx))
        A_new = A_new[mask, :]

        # update the row_idx and rest_of_rows
        row_idx = tl.index_update(row_idx, i, rest_of_rows[max_row_idx])
        rest_of_rows = rest_of_rows[tl.tensor(mask, dtype=tl.int64)]
        i = i + 1

    row_idx = tl.tensor(row_idx, dtype=tl.int64)
    inverse = tl.solve(
        A[row_idx, :], tl.eye(tl.shape(A[row_idx, :])[0], **tl.context(A))
    )
    row_idx = tl.to_numpy(row_idx)

    return row_idx, inverse

In [27]:
(10,) + (slice(None, None, None),) + (20,)

(10, slice(None, None, None), 20)

In [47]:
tensor = tl.tensor(np.arange(5**3).reshape(5,5,5))
rank = [1, 3, 3, 1]
factors = tensor_train_cross(tensor, rank)

[[()]]
[[(4, 2), (4, 3), (4, 0)], [(0,), (3,), (1,)], None]


In [36]:
tensor

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24]])

In [49]:
row_idx = [[()]]
col_idx = [[(4, 2), (4, 3), (4, 0)], [(0,), (3,), (1,)], None]
tensor_order = 2
k = 0
idx = (slice(None, None, None),) + tuple(zip(*col_idx[k]))

idx = tuple(idx)
print(idx)
tensor[idx]

(slice(None, None, None), (4, 4, 4), (2, 3, 0))


array([[ 22,  23,  20],
       [ 47,  48,  45],
       [ 72,  73,  70],
       [ 97,  98,  95],
       [122, 123, 120]])

In [41]:
tensor

array([[[  0,   1,   2,   3,   4],
        [  5,   6,   7,   8,   9],
        [ 10,  11,  12,  13,  14],
        [ 15,  16,  17,  18,  19],
        [ 20,  21,  22,  23,  24]],

       [[ 25,  26,  27,  28,  29],
        [ 30,  31,  32,  33,  34],
        [ 35,  36,  37,  38,  39],
        [ 40,  41,  42,  43,  44],
        [ 45,  46,  47,  48,  49]],

       [[ 50,  51,  52,  53,  54],
        [ 55,  56,  57,  58,  59],
        [ 60,  61,  62,  63,  64],
        [ 65,  66,  67,  68,  69],
        [ 70,  71,  72,  73,  74]],

       [[ 75,  76,  77,  78,  79],
        [ 80,  81,  82,  83,  84],
        [ 85,  86,  87,  88,  89],
        [ 90,  91,  92,  93,  94],
        [ 95,  96,  97,  98,  99]],

       [[100, 101, 102, 103, 104],
        [105, 106, 107, 108, 109],
        [110, 111, 112, 113, 114],
        [115, 116, 117, 118, 119],
        [120, 121, 122, 123, 124]]])

In [55]:
import numpy as np
from numpy.linalg import svd, norm

def initialize_tt(tensor, max_rank):
    """
    Initialize TT decomposition using random cores.
    """
    shape = tensor.shape
    d = len(shape)
    ranks = [1] + [min(max_rank, shape[i]) for i in range(1, d)] + [1]
    tt_cores = []
    
    for i in range(d):
        core_shape = (ranks[i], shape[i], ranks[i+1])
        tt_cores.append(np.random.randn(*core_shape))
    
    return tt_cores

def compute_left_contraction(tt_cores, k, tensor_shape):
    """
    Compute the left contraction up to core k.
    """
    left_contract = np.eye(tt_cores[0].shape[0])  # Start with identity for contraction
    for i in range(k):
        left_core = tt_cores[i]
        left_contract = np.tensordot(left_contract, left_core, axes=[-1, 0])
        left_contract = left_contract.reshape(-1, tensor_shape[i])  # Reshape for next contraction
    return left_contract

def compute_right_contraction(tt_cores, k, tensor_shape):
    """
    Compute the right contraction from core k+1.
    """
    right_contract = np.eye(tt_cores[-1].shape[-1])  # Start with identity for contraction
    for i in range(len(tt_cores) - 1, k, -1):
        right_core = tt_cores[i]
        right_contract = np.tensordot(right_core, right_contract, axes=[-1, 0])
        right_contract = right_contract.reshape(tensor_shape[i], -1)  # Reshape for next contraction
    return right_contract

def flatten_tensor(left_contract, right_contract, tensor, mode_dim):
    """
    Flatten the tensor for matrix representation (cross approximation).
    """
    full_contract = np.tensordot(left_contract, right_contract, axes=0)
    flat_tensor = np.tensordot(tensor, full_contract, axes=mode_dim)
    return flat_tensor.reshape(-1, flat_tensor.shape[-1])  # Matrix format

def cross_approximation(matrix, max_rank):
    """
    Perform cross approximation or SVD on the matrix.
    """
    u, s, vh = svd(matrix, full_matrices=False)
    rank = min(len(s), max_rank)
    truncated_core = u[:, :rank] @ np.diag(s[:rank]) @ vh[:rank, :]
    return truncated_core

def truncate_core(core, max_rank):
    """
    Truncate a TT core using SVD.
    """
    unfolded = core.reshape(core.shape[0] * core.shape[1], -1)
    u, s, vh = svd(unfolded, full_matrices=False)
    rank = min(len(s), max_rank)
    truncated_core = u[:, :rank] @ np.diag(s[:rank]) @ vh[:rank, :]
    return truncated_core

def tensor_train_dmrg_cross(tensor, max_rank, tol):
    """
    Perform Tensor Train DMRG Cross Algorithm.
    """
    shape = tensor.shape
    tt_cores = initialize_tt(tensor, max_rank)
    
    converged = False
    while not converged:
        max_error = 0
        for k in range(len(tt_cores)):
            # Compute left and right contractions
            left_contract = compute_left_contraction(tt_cores, k, shape)
            right_contract = compute_right_contraction(tt_cores, k, shape)
            
            # Flatten tensor and compute cross approximation
            mode_dim = (list(range(k)), list(range(k + 1, len(tt_cores))))
            flat_tensor = flatten_tensor(left_contract, right_contract, tensor, mode_dim)
            updated_core = cross_approximation(flat_tensor, max_rank)
            
            # Update and truncate the core
            tt_cores[k] = truncate_core(updated_core, max_rank)
        
            # Calculate error for convergence
            approx_tensor = reconstruct_tensor(tt_cores)
            error = norm(tensor - approx_tensor) / norm(tensor)
            max_error = max(max_error, error)
        
        if max_error < tol:
            converged = True
    
    return tt_cores

def reconstruct_tensor(tt_cores):
    """
    Reconstruct the full tensor from TT cores.
    """
    tensor = tt_cores[0]
    for i in range(1, len(tt_cores)):
        tensor = np.tensordot(tensor, tt_cores[i], axes=[-1, 0])
    return tensor.squeeze()


In [56]:
# Define a 3D tensor
tensor = np.random.randn(10, 10, 10)

# Set maximum TT rank and tolerance
max_rank = 5
tol = 1e-6

# Decompose tensor into TT format
tt_cores = tensor_train_dmrg_cross(tensor, max_rank, tol)

# Reconstruct the tensor and verify the approximation
approx_tensor = reconstruct_tensor(tt_cores)
error = norm(tensor - approx_tensor) / norm(tensor)
print(f"Relative approximation error: {error:.6f}")


ValueError: shape-mismatch for sum